In [6]:
from flask import Flask, request, jsonify, Response
from flask_cors import CORS
import zipfile
from zipfile import ZipFile
import os
import json
import pandas as pd
import numpy as np
import time 

In [7]:
# 폴더안에있는 Json파일 전부
def jsonzip(od_path):
    with zipfile.ZipFile(od_path, 'r') as obj:
        obj.namelist()		# zip파일 내의 파일 이름을 가진 list 반환
        obj.extractall("C:/datazip/")

 
def dataframe(od_path) :
    temp = od_path
    path = temp.split(".")[0]
    filename = od_path.split("/")[2].split(".")[0]
    list = os.listdir(path)
    json_file_list = [path+'/'+ file for file in list if file.endswith('.json')]
    
    imagePath = [] #이미지 파일명 저장 리스트
    label=[] #분류 라벨 값 저장 리스트
    points_1_x=[] #boundary box x,y좌표 리스트
    points_1_y=[] #boundary box x,y좌표 리스트
    points_2_x=[] #boundary box x,y좌표 리스트
    points_2_y=[] #boundary box x,y좌표 리스트
    errorFile =[] #에러 파일 검출 리스트
    shapes_list = [] 
    imageHeight = []
    imageWidth = []
    ratio = []
    wide = []
    bwide = []

    for i in json_file_list: #json_file_list의 하나의 인덱스 마다 i라는 변수로 사용한다.
        with open(i,'r') as j: #
            try:
                contents=json.loads(j.read())  # open : r - 읽기모드, w-쓰기모드, a-추가모드
                for k in range(len(contents['shapes'])): # contents라는 json객체 안에 여러 객체가 있을 경우를 대비해 반복문안에 shapes의 길이로 반복한다
                                                        # shapes의 내용이 2개면 2번 반복하는 형태 --> 변수는 k사용
                    print(contents['shapes'][k]) # 결과 확인 및 데이터 확인                 
                    shapes_list.append(len(contents['shapes']))
                    imagePath.append(str((contents['imagePath']).replace('.jpg',""))) # 따로 구별하는 변수가 없어서 이미지 경로에 확장자만 제거하여 사용
                    label.append(contents['shapes'][k]['label']) # label은 분리수거 되는 분류 클래스로 shapes->label 키 값으로 접근
                    points_1_x.append(contents['shapes'][k]['points'][0][0])
                    points_1_y.append(contents['shapes'][k]['points'][0][1])
                    points_2_x.append(contents['shapes'][k]['points'][1][0])
                    points_2_y.append(contents['shapes'][k]['points'][1][1])
                    imageHeight.append(contents['imageHeight'])
                    imageWidth.append(contents['imageWidth'])
            except:
                errorFile.append(i) # 크기가 0이거나, 파일 불러올 때 오류를 유발하는 파일명 errorFile리스트에 담는다.

    # 데이터 프레임 만들기
    # 딕셔너리 형태로 리스트를 묶어서 넣어준다.
    points_1_x=np.round(points_1_x).astype('int')
    points_1_y=np.round(points_1_y).astype('int')
    points_2_x=np.round(points_2_x).astype('int')
    points_2_y=np.round(points_2_y).astype('int')
    for i in range(len(imagePath)) :
        ratio.append((((points_2_x[i] - points_1_x[i])* (points_2_y[i] - points_1_y[i]))/( imageHeight[i] * imageWidth[i]))*100)
        print("Ratio for image", imagePath[i], ":", ratio)

    # 면적구하기
    for j in range(len(imagePath)) :
        wide.append(imageHeight[j]*imageWidth[j])
        wide.append((points_2_x[j] - points_1_x[j])*(points_2_y[j] - points_1_y[j]))
        print(wide[j], bwide[j])


    ratio=np.round(ratio,0)
    data=pd.DataFrame({"imagePath":imagePath,
                        "label":label,
                        "Shape_num":shapes_list,
                        "point1_x":points_1_x,
                        "point1_y":points_1_y,
                        "point2_x":points_2_x,
                        "point2_y":points_2_y,
                        "imageHeight":imageHeight,
                        "imageWidth":imageWidth,
                        "ratio" : ratio,
                        "wide" : wide,
                        "bwide" : bwide
                        })
    data.to_csv(f"{path}/{filename}.csv",index=False)
    return f"{path}/{filename}.csv"

In [3]:
data = pd.read_csv("C:/datazip/testzip/testzip.csv", index_col=False)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/datazip/testzip/testzip.csv'

In [ ]:
data['imagePath'] = data['imagePath'].astype("object")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   imagePath    3 non-null      object 
 1   label        3 non-null      object 
 2   Shape_num    3 non-null      int64  
 3   point1_x     3 non-null      int64  
 4   point1_y     3 non-null      int64  
 5   point2_x     3 non-null      int64  
 6   point2_y     3 non-null      int64  
 7   imageHeight  3 non-null      int64  
 8   imageWidth   3 non-null      int64  
 9   ratio        3 non-null      float64
dtypes: float64(1), int64(7), object(2)
memory usage: 372.0+ bytes


In [8]:
import pandas as pd 
import pymysql 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

# C:/datazip/testzip.zip
def file_db(od_path) : 

    conn = pymysql.connect(host='project-db-cgi.smhrd.com',port=3307, user = 'cgi_3_230524_1', password='smhrd1', db = 'cgi_3_230524_1',charset = 'utf8') 
    curs = conn.cursor(pymysql.cursors.DictCursor)


    path = str(od_path.split(".")[0])
    table_name = str(od_path.split("/")[2].split(".")[0])
    
    sql = f'''CREATE TABLE {table_name}_table( 
    num int(15) not null auto_increment primary key,
    imagePath varchar(500) not null, 
    label varchar(500) not null , 
    Shape_num  int(15) not null ,
    point1_x int(15) not null ,
    point1_y int(15) not null ,
    point2_x int(15) not null ,
    point2_y int(15) not null ,
    imageHeight int(15) not null ,
    imageWidth int(15) not null ,
    ratio int(15) not null
    ) 
    ''' 
    curs.execute(sql) 

    stores_info = pd.read_csv(f"{path}/{table_name}.csv" ,index_col=False)

    for index, row in stores_info.iterrows():
        tu = (
            row.num,
            row.imagePath,
            row.label,
            int(row.Shape_num),
            float(row.point1_x),
            float(row.point1_y),
            float(row.point2_x),
            float(row.point2_y),
            int(row.imageHeight),
            int(row.imageWidth),
            float(row.ratio)
        )
        curs.execute(
            f"""INSERT IGNORE INTO {table_name}_table (num, imagePath, label, Shape_num, point1_x, point1_y, point2_x, point2_y, imageHeight, imageWidth, ratio) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)""",
            tu
        )
        
    # 결과 저장 및 데이터베이스 종료
    conn.commit()
    conn.close()

In [9]:
# 플라스크 실행 코드

app = Flask(__name__)
CORS(app)

# 클릭했을때 반환해주는거잖아
@app.route('/test', methods=['POST'])
def handle_ajax_request():
    time.sleep(1)
    od_path = request.form.get('od_path')  # Ajax 요청에서 'od_path' 값을 가져옵니다.
    print(od_path)

    jsonzip(od_path)
    path = dataframe(od_path)
    # 응답을 반환합니다.

    data =pd.read_csv(path,index_col=False)
    data['imagePath'] = data['imagePath'].astype("object")
    print(path)
    # response = data.to_json(date_format="cp949", orient='index', indent=4)
    # param = response.get_json()
    # return jsonify(data.to_dict())
    return Response(data.to_json(orient="records"), mimetype='application/json')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


C:/datazip/datatest.zip


127.0.0.1 - - [09/Jun/2023 11:40:03] "POST /test HTTP/1.1" 200 -


{'label': 'paper', 'points': [[859.9496220000002, 1663.1999999999998], [1515.1498740000002, 2419.2]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[861.3333333333334, 1683.3333333333333], [1493.3333333333333, 2444.6666666666665]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'plastic', 'points': [[778.0498739999999, 1571.848992], [1433.250126, 2255.397984]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'vinyl', 'points': [[815.85063, 1505.6999999999998], [1549.800378, 2491.648992]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[837.90126, 1493.1000000000001], [1307.2502519999998, 2160.9]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[866.25126, 1486.797984], [1598.888888888889, 2498.8888888888887]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'can', 'points': [[800.0993700000001, 1675.8020

...

{'label': 'paper', 'points': [[859.9496220000002, 1663.1999999999998], [1515.1498740000002, 2419.2]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[861.3333333333334, 1683.3333333333333], [1493.3333333333333, 2444.6666666666665]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'plastic', 'points': [[778.0498739999999, 1571.848992], [1433.250126, 2255.397984]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'vinyl', 'points': [[815.85063, 1505.6999999999998], [1549.800378, 2491.648992]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[837.90126, 1493.1000000000001], [1307.2502519999998, 2160.9]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[866.25126, 1486.797984], [1598.888888888889, 2498.8888888888887]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'can', 'points': [[800.0993700000001, 1675.8020

127.0.0.1 - - [09/Jun/2023 12:44:50] "POST /test HTTP/1.1" 200 -


{'label': 'paper', 'points': [[859.9496220000002, 1663.1999999999998], [1515.1498740000002, 2419.2]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[861.3333333333334, 1683.3333333333333], [1493.3333333333333, 2444.6666666666665]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'plastic', 'points': [[778.0498739999999, 1571.848992], [1433.250126, 2255.397984]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'vinyl', 'points': [[815.85063, 1505.6999999999998], [1549.800378, 2491.648992]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[837.90126, 1493.1000000000001], [1307.2502519999998, 2160.9]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[866.25126, 1486.797984], [1598.888888888889, 2498.8888888888887]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'can', 'points': [[800.0993700000001, 1675.8020

127.0.0.1 - - [09/Jun/2023 12:45:11] "POST /test HTTP/1.1" 200 -


{'label': 'paper', 'points': [[859.9496220000002, 1663.1999999999998], [1515.1498740000002, 2419.2]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[861.3333333333334, 1683.3333333333333], [1493.3333333333333, 2444.6666666666665]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'plastic', 'points': [[778.0498739999999, 1571.848992], [1433.250126, 2255.397984]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'vinyl', 'points': [[815.85063, 1505.6999999999998], [1549.800378, 2491.648992]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[837.90126, 1493.1000000000001], [1307.2502519999998, 2160.9]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'paper', 'points': [[866.25126, 1486.797984], [1598.888888888889, 2498.8888888888887]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}
{'label': 'can', 'points': [[800.0993700000001, 1675.8020